# seaMass SWATH demo
This notebook illustrates how to load seaMass input/output on SWATH data. Firstly, we need to import the relevant modules...

In [1]:
using Plots; plotly(size=(980,350))
using Interact
import SeaMass

INFO: Precompiling module Plots.
    floattype(Type{#T<:FixedPointNumbers.Fixed}) at /home/awd/.julia/v0.4/FixedPointNumbers/src/fixed.jl:16
is ambiguous with: 
    floattype(Type{FixedPointNumbers.FixedPoint{#T<:Union{Int8, UInt16, Int16, UInt8}, #f<:Any}}) at /home/awd/.julia/v0.4/FixedPointNumbers/src/FixedPointNumbers.jl:89.
To fix, define 
    floattype(Type{FixedPointNumbers.Fixed{_<:Union{Int8, Int16}, #f<:Any}})
before the new definition.
    floattype(Type{#T<:FixedPointNumbers.UFixed}) at /home/awd/.julia/v0.4/FixedPointNumbers/src/ufixed.jl:14
is ambiguous with: 
    floattype(Type{FixedPointNumbers.FixedPoint{#T<:Union{Int8, UInt16, Int16, UInt8}, #f<:Any}}) at /home/awd/.julia/v0.4/FixedPointNumbers/src/FixedPointNumbers.jl:89.
To fix, define 
    floattype(Type{FixedPointNumbers.UFixed{_<:Union{UInt16, UInt8}, #f<:Any}})
before the new definition.
INFO: Precompiling module Interact.
    push!(DataStructures.Accumulator{#T<:Any, #V<:Number}, #T<:Any) at /home/awd/.julia/v0

INFO: Precompiling module HDF5.
INFO: Precompiling module PyCall.
INFO: Installing scipy.interpolate via the Conda scipy package...


Fetching package metadata .........
Solving package specifications: .

Package plan for installation in environment /home/awd/.julia/v0.4/Conda/deps/usr:

The following NEW packages will be INSTALLED:

    libgfortran: 3.0.0-1           
    scipy:       0.19.0-np112py27_0

libgfortran-3. 100% |###############################| Time: 0:00:01 251.85 kB/s
scipy-0.19.0-n 100% |###############################| Time: 0:01:26 426.02 kB/s  | Time: 0:00:01 395.72 kB/s2 459.86 kB/sipy-0.19.0-n   5% |#                              | Time: 0:00:04 455.68 kB/s% |##                             | Time: 0:00:06 460.96 kB/s                  | Time: 0:00:08 467.23 kB/s  | Time: 0:00:09 474.03 kB/s1 472.84 kB/sipy-0.19.0-n  17% |#####                          | Time: 0:00:13 467.79 kB/s% |######                         | Time: 0:00:15 463.46 kB/s                  | Time: 0:00:17 461.61 kB/s  | Time: 0:00:18 465.95 kB/s0 462.56 kB/sipy-0.19.0-n  28% |########                       | Time: 0:00:22 462.48 k

In [12]:
reload("SeaMass")

### 1. Display spectra from an mzMLb file

In [73]:
mzmlSpectra = SeaMass.MzmlSpectrum[]
maxSpectraIntensity = 0.0

for i = 1:100
    #push!(mzmlSpectra, SeaMass.MzmlSpectrum(
    #        "../data/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500.mzMLb", 2 + (i - 1) * 51))    
    push!(mzmlSpectra, SeaMass.MzmlSpectrum(
            "../data/3.smb2mzmlb/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500.mzMLb", 7062 + i))
    maxIntensity = maximum(mzmlSpectra[i].intensities)
    maxSpectraIntensity = (maxSpectraIntensity > maxIntensity) ? maxSpectraIntensity : maxIntensity
end

@manipulate for i = slider(1:100, label="Spectrum", value=63, continuous_update=true)
    plot(mzmlSpectra[i].mzs, mzmlSpectra[i].intensities, label="mzML input", m=2)
    plot!(xlabel = "m/z (Th)", ylabel = "intensity",
        xlims=(602,605), ylims=(-0.05*maxSpectraIntensity,maxSpectraIntensity))
end

Interact.Slider{Int64}(Signal{Int64}(63, nactions=1),"Spectrum",63,1:100,"horizontal",true,"d",true)

### 2. Compare the spectra from the SMB file

In [74]:
@manipulate for i = slider(1:100, label="Spectrum", value=63, continuous_update=true)
    smbSpectrum = SeaMass.SmbSpectrum(
        "../data/0.mzmlb2smb/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500.p-411-25.smb", i)

    plot(smbSpectrum.binLocations, vcat(smbSpectrum.counts / smbSpectrum.exposure, 0.0), line=:steppost, label="SMB input", m=2)
    plot!(mzmlSpectra[i].mzs, mzmlSpectra[i].intensities, label="mzML input", m=2)
    plot!(xlabel = "m/z (Th)", ylabel = "ion density",
        xlims=(602,605),
        ylims=(-0.05*maxSpectraIntensity,maxSpectraIntensity))
end

Interact.Slider{Int64}(Signal{Int64}(63, nactions=1),"Spectrum",63,1:100,"horizontal",true,"d",true)

### 3. Compare output of 1D seaMass to 1D SMI file

In [ ]:
smiSpectrum = SeaMass.SmiSpectrum(
    "../data/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500__index3163.pos_411.25.smi", 1)
smiBinWidths = smiSpectrum.locations[2:end] - smiSpectrum.locations[1:end-1]

smoSpectrum = SeaMass.SmoSpectrum(
    "../data/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500__index3163.pos_411.25.smo1D", 1)
binnedSmoSpectrum = SeaMass.BinnedSmoSpectrum(smoSpectrum)
sampledSmoSpectrum = SeaMass.MzmlSpectrum(smoSpectrum, 100000)

plot(smiSpectrum.locations, vcat(smiSpectrum.counts  ./ smiBinWidths, 0.0), line=:steppost, label="SMI input")
plot!(binnedSmoSpectrum.locations, vcat(binnedSmoSpectrum.counts / binnedSmoSpectrum.binWidth, 0.0),
      line=:steppost, label="binned reconstruction")
plot!(sampledSmoSpectrum.mzs, sampledSmoSpectrum.intensities, label="sampled reconstruction")
plot!(xlabel = "m/z (Th)", ylabel = "ion density (counts per Th)")

#### 3.1. Animation

In [ ]:
smoFilenames = filter!(r"^P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500__index3163\.pos_411.25\..*smo1D$",
    readdir("../data/"))

@manipulate for i=slider(0:length(smoFilenames)-1, label="Iteration", value=0,
        continuous_update=true)
    smoSpectrum = SeaMass.SmoSpectrum("../data/" * smoFilenames[i+1], 1)
    sampledSmoSpectrum = SeaMass.MzmlSpectrum(smoSpectrum, 10000)
    
    plot(smiSpectrum.locations, vcat(smiSpectrum.counts  ./ smiBinWidths, 0.0), line=:steppost, label="SMI input")
    plot!(sampledSmoSpectrum.mzs, sampledSmoSpectrum.intensities, label="sampled reconstruction")
    plot!(xlabel = "m/z (Th)", ylabel = "ion density (counts per Th)")
    plot!(xlims = (602.,605.), ylims = (-10000.,120000.))
end

### 4. Compare output of 2D seaMass to 1D SMI file

In [ ]:
smiSpectra = SeaMass.SmiSpectrum[]
maxSpectraBinCounts = 0.0
for i = 1:100
    smiSpectrum = SeaMass.SmiSpectrum(
        "../data/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500.pos_411.25.smi", i)
    smiBinWidths = smiSpectrum.locations[2:end] - smiSpectrum.locations[1:end-1]
    smiSpectrum.counts = vcat(smiSpectrum.counts  ./ smiBinWidths, 0.0)
    maxBinCounts = maximum(smiSpectrum.counts)
    maxSpectraBinCounts = (maxSpectraBinCounts > maxBinCounts) ? maxSpectraBinCounts : maxBinCounts
    push!(smiSpectra, smiSpectrum)
end

@manipulate for i = slider(1:100, label="Spectrum", value=63, continuous_update=true)
    smoSpectrum = SeaMass.SmoSpectrum(
        "../data/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500.pos_411.25.smo1D", i)
    binnedSmoSpectrum = SeaMass.BinnedSmoSpectrum(smoSpectrum)
    sampledSmoSpectrum = SeaMass.MzmlSpectrum(smoSpectrum, 100000)

    plot(smiSpectra[i].locations, smiSpectra[i].counts, line=:steppost, label="SMI input")
    plot!(binnedSmoSpectrum.locations, vcat(binnedSmoSpectrum.counts / binnedSmoSpectrum.binWidth, 0.0),
      line=:steppost, label="binned reconstruction")
    plot!(sampledSmoSpectrum.mzs, sampledSmoSpectrum.intensities, label="sampled reconstruction")
    plot!(xlabel = "m/z (Th)", ylabel = "ion density (counts per Th)",
        xlims=(602,605), ylims=(-0.05*maxSpectraBinCounts,maxSpectraBinCounts))
end